<a href="https://colab.research.google.com/github/Padmaja004/trend-seasonality/blob/main/time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Here's a Python code that calculates trend and seasonality using the provided formulas and plots the combined trend and seasonality using the AirPassengers dataset:


import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Load AirPassengers dataset
data = pd.read_csv("/content/sales.csv")
data.index = pd.to_datetime(data.index)

# Calculate trend using odd moving average
def calculate_trend_odd_moving_average(time_series, q):
    trend = []
    window_size = 2 * q + 1
    for i in range(len(time_series)):
        if i < q:
            trend.append(sum(time_series[:i+1]) / (i + 1))
        elif i >= len(time_series) - q:
            trend.append(sum(time_series[i:]) / (len(time_series) - i))
        else:
            trend.append(sum(time_series[i-q:i+q+1]) / window_size)
    return trend

# Calculate trend using weighted average
def calculate_trend_weighted_average(time_series, q):
    trend = []
    for i in range(len(time_series)):
        if i < q:
            trend.append(time_series[i])
        elif i >= len(time_series) - q:
            trend.append(time_series[i])
        else:
            window_sum = 0.5 * time_series[i-q] + sum(time_series[i-q+1:i+q]) + 0.5 * time_series[i+q]
            trend.append(window_sum / (2 * q + 1))
    return trend

# Calculate seasonality
def calculate_seasonality(time_series, trend, d):
    seasonality = []
    weights = []
    n = len(time_series)
    for k in range(n):
        weight_sum = 0
        count = 0
        for j in range(-q, q+1):
            if 0 <= k + j * d < n:
                weight_sum += time_series[k + j * d] - trend[k + j * d]
                count += 1
        weights.append(weight_sum / count)
    avg_weight = sum(weights) / len(weights)
    for k in range(n):
        seasonality.append(weights[k] - avg_weight)
    return seasonality

# Parameters
q = 2  # window size parameter
d = 12  # seasonality period

# Calculate trend and seasonality
trend_odd_moving_average = calculate_trend_odd_moving_average(data['Margin'], q)
trend_weighted_average = calculate_trend_weighted_average(data['Margin'], q)
seasonality = calculate_seasonality(data['Margin'], trend_weighted_average, d)

# Combine trend and seasonality
combined = [t + s for t, s in zip(trend_weighted_average, seasonality)]

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(data['Margin'], label='Time Series', alpha=0.5)
plt.plot(trend_weighted_average, label='Trend', color='orange')
plt.plot(seasonality, label='Seasonality', color='green')
plt.plot(combined, label='Trend + Seasonality', color='red')
plt.title('Air Passengers Time Series Decomposition')
plt.xlabel('Time')
plt.ylabel('Margin')
plt.legend()
plt.show()


